In [1]:
import cv2
import numpy as np

In [10]:
# Load Yolo

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  

# dnn is deep neural network

classes = []   

# in classes you can specify objects that you want to detect 

# classes=['car','person','bicycle'] 

with open("coco.names", "r") as f:      

    classes = [line.strip() for line in f.readlines()]

#print the name of classes
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [8]:
# Loading classes from coco file

layer_names = net.getLayerNames()

output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# with output layer we get detection of objects.

colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [9]:
video=cv2.VideoCapture(0)

while True:

    ret,img=video.read()
    height, width, channels = img.shape
    # Detecting objects

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(output_layers)
    # Showing informations on the screen

    class_ids = []

    confidences = []

    boxes = []

    for out in outs:

        for detection in out:

            scores = detection[5:]

            class_id = np.argmax(scores)

            confidence = scores[class_id]

            if confidence > 0.5:

                # Object detected

                center_x = int(detection[0] * width)

                center_y = int(detection[1] * height)

                w = int(detection[2] * width)

                h = int(detection[3] * height)

                # Rectangle coordinates

                x = int(center_x - w / 2)

                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])

                confidences.append(float(confidence))

                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    #print(indexes)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):

        if i in indexes:

            x, y, w, h = boxes[i]

            label = str(classes[class_ids[i]])

            color = colors[i]

            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)    

    #frame = cv2.resize(frame, None, fx=2, fy=2)
    cv2.imshow("live cam", img)
    
    if cv2.waitKey(30) & 0xFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()